In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("https://github.com/LeoNefesch/data_analysis/blob/main/datasets/telecom_churn.csv")
df.head()

In [ ]:
#показывать не 60*20, а 100*100 (строк*столбцов)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
#Размер нашего датафрейма (строк, столбцов)
df.shape

In [ ]:
#Названия столбцов
df.columns

In [ ]:
#Основная информация по датафрейму (тип данных, есть ли пустые значения)
df.info()

In [ ]:
#Заменить тип данных в столбце (у нас с bool на int64)
df['Churn'] = df['Churn'].astype('int64')

In [ ]:
#статистика по числовым признакам (int64, float64)
df.describe()

In [ ]:
#статистика по нечисловым признакам
df.describe(include=['object', 'bool'])

In [ ]:
#Распределение данных по целевому признаку Churn (отток клиентов). 0 - клиент лояльный, 1 - клиент ушёл
df['Churn'].value_counts()

In [ ]:
#распределение пользователей по признаку Area code. Смотрим абсолютные частоты.
df['Area code'].value_counts(normalize=True)

In [ ]:
#отсортируем датафрейм по одному столбцу, по убыванию, выведем первые 5 записей
df.sort_values(by='Total day charge', ascending=False).head()

In [ ]:
#Сортировка по группе столбцов
df.sort_values(by=['Churn','Total day charge'], ascending=[True,False]).head()

In [ ]:
#какова доля нелояльных пользователей?
df['Churn'].mean()

In [ ]:
#ср.значения числовых признаков среди нелояльных пользователей
df[df['Churn']==1].mean()

In [ ]:
#Средняя длительность телефон.разговора нелояльного пользователя днём
df[df['Churn']==1]['Total day minutes'].mean()

In [ ]:
#макс.длина международного звонка лоял.пользователя, не пользующегося международным роумингом
df[(df['Churn']==0)&(df['International plan']=='No')]['Total intl minutes'].max()

In [ ]:
#срез строк с 0 по 5 включительно и указанных столбцов
df.loc[0:5, 'State':'Area code']

In [ ]:
#срез строк с 0 по 5 не включительно (первые 5) и столбцов с 0 по 3 не включительно (первые 3)
df.iloc[0:5, 0:3]

In [ ]:
#первая строка df
df[:1]


In [ ]:
 #последняя строка df
 df[-1:]

In [ ]:
#применение функции к каждому столбцу - функция apply()
df.apply(np.max)

In [ ]:
#замена значений в каждой ячейке столбца, функция map с аргументом d - словарь {old_value: new_value}
d = {"No": False, "Yes": True}
df['International plan'] = df['International plan'].map(d)
df.head()

In [ ]:
#метод replace действует аналогично
df = df.replace({'Voice mail plan': d})
df.head()

In [ ]:
#Группировка данных в зависимости от значения Churn, вывод статистики по трём столбцам в каж.группе
columns_to_show = ['Total day minutes','Total eve minutes','Total night minutes']
df.groupby(['Churn'])[columns_to_show].describe(percentiles=[])

In [ ]:
#применим функцию agg, передав в кач-ве аргументов нужные нам стат.функции
df.groupby(['Churn'])[columns_to_show].agg([np.mean,np.std,np.min,np.max])

In [ ]:
#Таблица сопряженности по 2 признакам, функция crosstab, normalize=False -отн.частота,сколько раз встречается значение
pd.crosstab(df['Churn'], df['International plan'], normalize=False)

In [ ]:
#Таблица сопряжённости, заменим один признак (роуминг на голос.почту) и выведем абс.частоту (сумма=1)
pd.crosstab(df['Churn'], df['Voice mail plan'], normalize=True)

In [ ]:
#Сводная таблица: смотрим среднее число дневных, вечерних и ночных звонков для разных регионов
df.pivot_table(['Total day calls', 'Total eve calls', 'Total night calls'], ['Area code'], aggfunc='mean').head(10)

In [ ]:
#добавим в наш датафрэйм столбец с общим кол-вом звонков. total_calls - объект типа Series
total_calls = df['Total day calls'] + df['Total eve calls'] + df['Total night calls'] + df['Total intl calls']
df.insert(loc=len(df.columns), column='Total calls', value=total_calls)
df.head()

In [ ]:
#более простой способ вставить новый столбец (не создавая явно объект Series)
df['Total charge'] = df['Total day charge'] + df['Total eve charge'] + df['Total night charge'] + df['Total intl charge']
df.head()

In [ ]:
#сохраним полученный df в новую переменную
total_df = df
total_df.head()

In [ ]:
#удалим из df добавленные столбцы. axis=1, если удаляем столбцы
df = df.drop(['Total calls', 'Total charge'], axis=1)
df.head()

Оценим связь оттока с подключением международного роуминга

In [ ]:
pd.crosstab(df['Churn'], df['International plan'], margins=True)

In [ ]:
import seaborn as sns

In [ ]:
sns.catplot(x='International plan', hue='Churn', kind= 'count', data=df)

Из диаграммы видно, что доля нелояльных клиентов - почти половина из тех, кому подключили роуминг. Возможно, это из-за сильно возросших расходов на связь.

Обратим внимание на число обращений в сервисный центр

In [ ]:
pd.crosstab(df['Churn'], df['Customer service calls'], margins=True)

In [ ]:
sns.catplot(x='Customer service calls', hue='Churn', kind= 'count', data=df)

Видно, что начиная с четвёртого звонка в сервисный центра доля оттока клиентов возрастает, начиная с пятого звонка - преобладает.

Добавим в df бинарный признак - столбец с результатом сравнения "Звонок в сервисный центр" > 3 раз

In [ ]:
df['Many_service_calls'] = (df['Customer service calls']>3).astype('int')
pd.crosstab(df['Many_service_calls'], df['Churn'], margins=True)

In [ ]:
sns.catplot(x='Many_service_calls', hue='Churn', kind= 'count', data=df)

Объединим условия (подключение роуминга и звонки в сервисный центр) и построим сопряжённую таблицу

In [ ]:
pd.crosstab(df['Many_service_calls']&df['International plan'],df['Churn'])

Когда число звонков в сервисный центр больше 3 и подключен роуминг (и прогнозируя лояльность – в противном случае), можно ожидать около 85.8% правильных попаданий (False ошибаемся по двум параметрам всего (464 + 9 раз) из 3333 раз-14,2%).